# Using the EFD python client to query metrics from analysis_tools

**Description:** Demonstrate how to query metric values produced with [analysis_tools](https://github.com/lsst/analysis_tools)[analysis_tools](https://github.com/lsst/analysis_tools) using the EFD python client for the Sasquatch environment. 

More information at https://sasquatch.lsst.io/

**Contact authors:** Keith Bechtol

**Last verified to run:** 2023-09-13

**LSST Science Piplines version:** w_2023_34

**Container Size:** any

**Location:** This notebook was developed on the S3DF cluster at the USDF.

In [1]:
!eups list -s | grep lsst_distrib

lsst_distrib          g4213664e8e+d053631def 	current w_2023_44 setup


Currently, the analysis_tools metrics are dispatched to the USDF dev Sasquatch environment, indicated with the `usdfdev_efd` alias, and are located in the `lsst.dm` database.

In [2]:
from lsst_efd_client import EfdClient
client = EfdClient("usdfdev_efd", db_name="lsst.dm")

The topics correspond to individual analysis tools, e.g., as specified in the pipeline yaml file [coaddQualityCore.yaml](https://github.com/lsst/analysis_tools/blob/18877759150ddbd865aecc4be00199d1975baee4/pipelines/coaddQualityCore.yaml#L9). Consider `e1Diff` as an example.

In [3]:
topics = await client.get_topics()
for t in topics: print(t)

lsst.dm.ap12PsfSky
lsst.dm.blendMetrics
lsst.dm.e1Diff
lsst.dm.e1DiffScatterMetric
lsst.dm.e2Diff
lsst.dm.e2DiffScatterMetric
lsst.dm.isolatedDeblenderMetrics
lsst.dm.matchedRefCModelFluxChiMetric
lsst.dm.matchedRefCModelMagChi
lsst.dm.matchedRefCModelMagDiff
lsst.dm.matchedRefCModelMagDiffMetric
lsst.dm.matchedRefMagChiMetric
lsst.dm.matchedRefPositionXChi
lsst.dm.matchedRefPositionXChiMetric
lsst.dm.matchedRefPositionXDiff
lsst.dm.matchedRefPositionXDiffMetric
lsst.dm.matchedRefPositionYChi
lsst.dm.matchedRefPositionYChiMetric
lsst.dm.matchedRefPositionYDiff
lsst.dm.matchedRefPositionYDiffMetric
lsst.dm.parentDeblenderMetrics
lsst.dm.psfCModelSky
lsst.dm.shapeSizeFractionalDiff
lsst.dm.shapeSizeFractionalMetric
lsst.dm.skippedDeblenderMetrics
lsst.dm.skyFluxStatisticMetric
lsst.dm.stellarAstrometricRepeatability1
lsst.dm.stellarAstrometricRepeatability2
lsst.dm.stellarAstrometricRepeatability3
lsst.dm.stellarPhotometricRepeatability
lsst.dm.targetRefCatDeltaCModelScatterPlot
lsst.dm.

For a given topic, the fields correspond to metadata and metrics. As an example, the fields corresponding to metrics in the topic `e1Diff` where specified as keys in the `MetricMeasurementBundle` for the [E1Diff](https://github.com/lsst/analysis_tools/blob/18877759150ddbd865aecc4be00199d1975baee4/python/lsst/analysis/tools/atools/shapes.py#L141) class in analysis tools. 

In [4]:
await client.get_fields("lsst.dm.e1Diff")

['band',
 'dataset_tag',
 'dataset_type',
 'detector',
 'exposure',
 'g_highSNStars_count',
 'g_highSNStars_median',
 'g_highSNStars_sigmaMad',
 'g_lowSNStars_count',
 'g_lowSNStars_median',
 'g_lowSNStars_sigmaMad',
 'i_highSNStars_count',
 'i_highSNStars_median',
 'i_highSNStars_sigmaMad',
 'i_lowSNStars_count',
 'i_lowSNStars_median',
 'i_lowSNStars_sigmaMad',
 'id',
 'instrument',
 'patch',
 'physical_filter',
 'r_highSNStars_count',
 'r_highSNStars_median',
 'r_highSNStars_sigmaMad',
 'r_lowSNStars_count',
 'r_lowSNStars_median',
 'r_lowSNStars_sigmaMad',
 'reference_package',
 'reference_package_timestamp',
 'reference_package_version',
 'run',
 'run_timestamp',
 'skymap',
 'timestamp',
 'tract',
 'u_highSNStars_count',
 'u_highSNStars_median',
 'u_highSNStars_sigmaMad',
 'u_lowSNStars_count',
 'u_lowSNStars_median',
 'u_lowSNStars_sigmaMad',
 'visit',
 'y_highSNStars_count',
 'y_highSNStars_median',
 'y_highSNStars_sigmaMad',
 'y_lowSNStars_count',
 'y_lowSNStars_median',
 'y_lo

We can query a given topic using [InfluxQL syntax](https://docs.influxdata.com/influxdb/v1/query_language/explore-data). Notice that the columns in the returned DataFrame match the fields above.

In [5]:
query = '''SELECT * FROM "lsst.dm.e1Diff" WHERE time > now() - 100d '''
df = await client.influx_client.query(query)
df.columns.values

array(['band', 'dataset_tag', 'dataset_tag_1', 'dataset_type', 'detector',
       'exposure', 'g_highSNStars_count', 'g_highSNStars_median',
       'g_highSNStars_sigmaMad', 'g_lowSNStars_count',
       'g_lowSNStars_median', 'g_lowSNStars_sigmaMad',
       'i_highSNStars_count', 'i_highSNStars_median',
       'i_highSNStars_sigmaMad', 'i_lowSNStars_count',
       'i_lowSNStars_median', 'i_lowSNStars_sigmaMad', 'id', 'instrument',
       'patch', 'physical_filter', 'r_highSNStars_count',
       'r_highSNStars_median', 'r_highSNStars_sigmaMad',
       'r_lowSNStars_count', 'r_lowSNStars_median',
       'r_lowSNStars_sigmaMad', 'reference_package',
       'reference_package_timestamp', 'reference_package_version', 'run',
       'run_1', 'run_timestamp', 'skymap', 'skymap_1', 'timestamp',
       'tract', 'tract_1', 'u_highSNStars_count', 'u_highSNStars_median',
       'u_highSNStars_sigmaMad', 'u_lowSNStars_count',
       'u_lowSNStars_median', 'u_lowSNStars_sigmaMad', 'visit',
       'y_

Inspect the table. Each row corresponds to one `MetricMeasurementBundle` from analysis tools.

In [ ]:
df

Suppose that we are only interested in metrics with a given tag, e.g., `HSC/RC2`. We can refine our query as follows to select on field values.

In [6]:
query = '''SELECT * FROM "lsst.dm.e1Diff" WHERE time > now() - 100d AND "dataset_tag" = 'HSC/RC2' '''
df = await client.influx_client.query(query)
df

,band,dataset_tag,dataset_tag_1,dataset_type,detector,exposure,g_highSNStars_count,g_highSNStars_median,g_highSNStars_sigmaMad,g_lowSNStars_count,...,y_highSNStars_sigmaMad,y_lowSNStars_count,y_lowSNStars_median,y_lowSNStars_sigmaMad,z_highSNStars_count,z_highSNStars_median,z_highSNStars_sigmaMad,z_lowSNStars_count,z_lowSNStars_median,z_lowSNStars_sigmaMad
2023-09-28 07:00:00+00:00,,HSC/RC2,HSC/RC2,objectTableCore_metrics,,,6,0.004315,0.002616,4205,...,0.003277,6363,0.000432,0.004378,113,0.001143,0.003317,9538,0.000187,0.003557
2023-09-28 07:00:00+00:00,,HSC/RC2,HSC/RC2,objectTableCore_metrics,,,663,0.000804,0.002882,4650,...,0.002325,6444,0.000065,0.003154,1299,0.000844,0.002409,7951,0.000147,0.003317
2023-09-28 07:00:00+00:00,,HSC/RC2,HSC/RC2,objectTableCore_metrics,,,29,0.008620,0.012367,3441,...,0.000000,6309,0.000241,0.004255,39,0.001563,0.001888,9257,0.000150,0.003672
2023-09-28 07:00:00+00:00,,HSC/RC2,HSC/RC2,objectTableCore_metrics,,,6,0.004315,0.002616,4205,...,0.003277,6363,0.000432,0.004378,113,0.001143,0.003317,9538,0.000187,0.003557
2023-09-28 07:00:00+00:00,,HSC/RC2,HSC/RC2,objectTableCore_metrics,,,663,0.000804,0.002882,4650,...,0.002325,6444,0.000065,0.003154,1299,0.000844,0.002409,7951,0.000147,0.003317
2023-09-28 07:00:00+00:00,,HSC/RC2,HSC/RC2,objectTableCore_metrics,,,29,0.008620,0.012367,3441,...,0.000000,6309,0.000241,0.004255,39,0.001563,0.001888,9257,0.000150,0.003672
2023-11-16 08:00:00+00:00,,HSC/RC2,HSC/RC2,objectTableCore_metrics,,,6,0.002254,0.002776,4185,...,0.003140,6159,0.000445,0.004330,102,0.001190,0.003555,9363,0.000224,0.003557
2023-11-16 08:00:00+00:00,,HSC/RC2,HSC/RC2,objectTableCore_metrics,,,661,0.000870,0.002955,4451,...,0.002281,6163,0.000082,0.003021,1289,0.000826,0.002583,7595,0.000123,0.003311
2023-11-16 08:00:00+00:00,,HSC/RC2,HSC/RC2,objectTableCore_metrics,,,26,0.011203,0.009631,3090,...,0.000000,6252,0.000273,0.004230,37,0.001937,0.002435,9126,0.000085,0.003684
2023-11-23 08:00:00+00:00,,HSC/RC2,HSC/RC2,objectTableCore_metrics,,,13,0.002292,0.004778,4544,...,0.003216,5464,0.000637,0.005443,80,0.001803,0.003572,8987,0.000395,0.004594


Finally, access the specific metric values.

In [7]:
df.g_highSNStars_count.values

array([  6, 663,  29,   6, 663,  29,   6, 661,  26,  13, 648,  20,  11,
       754,  23])